# Homework 5: Lexicons and Distributional Semantics

This is due on **Friday, 11/10 (11pm)** 

## How to do this problem set

Most of these questions require writing Python code and computing results, and the rest of them have textual answers.  Write all the textual answers in this document, show the output of your experiment in this document, and implement the functions in the `python` files. 

Submit a PDF of thie .ipynb to Gradescope, and the .ipynb and all python files to Moodle.

The assignment has two parts:
 * In the first part, you will experiment with Turney's method to find word polarities in a twitter dataset, given some positive and negative seed words.
 * In the second part, you will experiment with distributional and vector semantics.

**Your Name: <>**

**List collaborators, and how you collaborated, here:** (see our [grading and policies page](http://people.cs.umass.edu/~brenocon/inlp2017/grading.html) for details on our collaboration policy).

* _Rheeya Uppal_ 

## Part 1: Lexicon semantics


Recall that PMI of a pair of words, is defined as:

$$PMI(x, y) = log\frac{ P(x, y) }{ P(x)P(y)}$$

The Turney mehod defines a word's polarity as:

$$Polarity(word) = PMI(word, positive\_word)−PMI(word, negative\_word)$$

where the joint probability $P(w, v)$ or, more specifically, $P(w\ NEAR\ v)$ is the probability of both being "near" each other.  We'll work with tweets, so it means: if you choose a tweet at random, what's the chance it contains both `w` and `v`?

(If you look at the Turney method as explained in the SLP3 chapter, the "hits" function is a count of web pages that contain at least one instance of the two words occurring near each other.)

The positive_word and negative_word terms are initially constructed by hand. For example: we might start with single positive word ('excellent') and a single negative word ('bad'). We can also have list of positive words ('excellent', 'perfect', 'love', ....) and list of negative words ('bad', 'hate', 'filthy',....)

If we're using a seed list of multiple terms, just combine them into a single symbol, e.g. all the positive seed words get rewritten to POS_WORD (and similarly for NEG_WORD).  This $P(w, POS\_WORD)$ effectively means the co-ocurrence of $w$ with any of the terms in the list.

For this assignment, we will use twitter dataset which has 349378 tweets. These tweets are in the file named `tweets.txt`. These are the tweets of one day and filtered such that each tweet contains at least one of the seed words we've selected.

## Question 1 (15 points)

The file `tweets.txt` contains around 349,378 tweets with one tweet per line.  It is a random sample of public tweets, which we tokenized with [twokenize.py's tokenizeRawTweetText()](https://github.com/myleott/ark-twokenize-py/blob/master/twokenize.py)). The text you see has a space between each token so you can just use `.split()` if you want.  We also filtered tweets to ones that included at least one term from one of these seed lists:
* Positive seed list: ["good", "nice", "love", "excellent", "fortunate", "correct", "superior"]
* Negative seed list: ["bad", "nasty", "poor", "hate", "unfortunate", "wrong", "inferior"]

Each tweet contains at least one positive or negative seed word. Take a look at the file (e.g. `less' and `grep'). Implement the Turney's method to calculate polarity scores of all words.

Some things to keep in mind:
* Ignore the seed words (i.e. don't calculate the polarity of the seed words).
* You may want to ignore the polarity of words beignning with `@` or `#`. 

We recommend that you write code in a python file, but it's up to you.

QUESTION: You'll have to do something to handle zeros in the PMI equation. Please explain your and justify your decision about this.

**Answer**

There might be certain words that occur alongside a positive word but not a negative word or vice-versa. So for this reason pseudocounts are required. I chose laplace smoothing with alpha=1.

$$PMI(x, y) = log\frac{ P(x, y) +1 }{ P(x)P(y)+|V|}$$
where |V| is the size of the vocaublary and alpha =1

## Question 2 (5 points)

Print the top 50 most-positive words (i.e. inferred positive words) and the 50 most-negative words.

Many of the words won't make sense.  Comment on at least two that do make sense, and at least two that don't.  Why do you think these are happening with this dataset and method?

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Write code to print words here
import  math
#import nltk
import operator
from collections import defaultdict
from sets import Set

pos_seedlist=["good", "nice", "love", "excellent", "fortunate", "correct", "superior"]
neg_seedlist=["bad", "nasty", "poor", "hate", "unfortunate", "wrong", "inferior"]

sentences=open('/home/twinkle/NLP/hw3/tweets.txt').read().strip().split("\n")
#one row
#condition to ignore
wordcounts=defaultdict(int)
pair_counts=defaultdict(int)
seedsum=defaultdict(float)
pmi=defaultdict(float)
polarity=defaultdict(float)
total=0
bow=Set()
words=[]
allwords=[]
i=0
i=0
#bla=[]
for sent in sentences:
    print i    
    i+=1
    temp=sent.split(' ')
    words+=temp
    setwords=Set(temp)
    bow=bow|setwords
    for word in setwords:
        word=word.lower()
        wordcounts[word]+=1
        condition=(word in pos_seedlist) or (word in neg_seedlist) or (word[0]=='@') or (word[0]=='#')  or (word[:5]=='https') or (word=='RT') or (word==':')
        if not condition:
            for seed in pos_seedlist:
                if sent.find(seed) != -1 :
                    pair_counts[(word,'pos')]+=1
                    seedsum['pos']+=pair_counts[(word,'pos')]
            for seed in neg_seedlist:
                if sent.find(seed) != -1 :
                    pair_counts[(word,'neg')]+=1
                    seedsum['neg']+=pair_counts[(word,'pos')]
total=seedsum['pos']+seedsum['neg']

pmi={}
print "DONE PAIR_COUNTS"
for key in pair_counts.keys():
    den=seedsum[key[1]]*wordcounts[key[0]]+len(pair_counts)
    if den==0:
        print seedsum[key[1]], wordcounts[key[0]],key
    print float(total*pair_counts[key]+1)/den
    pmi[key]=((total*pair_counts[key])+1)/den
#    ppmi[key]=max(math.log(pmi),0)
print "DONE PMI"



#polarity
for key in pair_counts.keys():
    polarity[key[0]]=pmi[(key[0],'pos')]-pmi[(key[0],'neg')]
    
print polarity
print"\n\n\n"
    
sorted_polarity1=sorted(polarity.items(),key=operator.itemgetter(1))
sorted_polarity2=sorted(polarity.items(),key=operator.itemgetter(1),reverse=True)
print "ACCENDING",sorted_polarity1[:20]
print "REVERSE",sorted_polarity2[:20]



### Textual answer here.

## Question 3 (5 points)

Now filter out all the words which have total count < 500, and then print top 50 polarity words and bottom 50 polarity words. 

Choose some of the words from both the sets of 50 words you got above which accoording to you make sense. Again please note, you will find many words which don't make sense. Do you think these results are better than the results you got in Question-1? Explain why.

In [4]:
# Write code to print words here


############for 500


for key in pair_counts.keys():
    if pair_counts[key]<500:
        pair_counts.pop(key,None)
    else:
        den=seedsum[key[1]]*wordcounts[key[0]]+len(pair_counts)
        if den==0:
            print seedsum[key[1]], wordcounts[key[0]],key
        pmi=(total*pair_counts[key]+1)/den
    #    ppmi[key]=max(math.log(pmi),0)
print "DONE PMI"


#polarity
for key in pair_counts.keys():
    polarity[key[0]]=pmi[(key[0],'pos')]-pmi[(key[0],'neg')]
    
print polarity
print"\n\n\n"
    
sorted_polarity1=sorted(polarity.items(),key=operator.itemgetter(1))
sorted_polarity2=sorted(polarity.items(),key=operator.itemgetter(1),reverse=True)
print "ACCENDING",sorted_polarity1[:20]
print "REVERSE",sorted_polarity2[:20]


### Textual answer here.

## Question 4 (5 points)

Even after filtering out words with count < 500, many top-most and bottom-most polarity don't make sense. Identify what kind of words these are and what can be done to filter them out. You can read some tweets in the file to see what's happening. 

### Textual answer here.

# Part-2: Distributional Semantics

## Cosine Similarity

Recall that, where $i$ indexes over the context types, cosine similarity is defined as follows. $x$ and $y$ are both vectors of context counts (each for a different word), where $x_i$ is the count of context $i$.

$$cossim(x,y) = \frac{ \sum_i x_i y_i }{ \sqrt{\sum_i x_i^2} \sqrt{\sum_i y_i^2} }$$

The nice thing about cosine similarity is that it is normalized: no matter what the input vectors are, the output is between 0 and 1. One way to think of this is that cosine similarity is just, um, the cosine function, which has this property (for non-negative $x$ and $y$). Another way to think of it is, to work through the situations of maximum and minimum similarity between two context vectors, starting from the definition above.

Note: a good way to understand the cosine similarity function is that the numerator cares about whether the $x$ and $y$ vectors are correlated. If $x$ and $y$ tend to have high values for the same contexts, the numerator tends to be big. The denominator can be thought of as a normalization factor: if all the values of $x$ are really large, for example, dividing by the square root of their sum-of-squares prevents the whole thing from getting arbitrarily large. In fact, dividing by both these things (aka their norms) means the whole thing can’t go higher than 1.

In this problem we'll work with vectors of raw context counts.  (As you know, this is not necessarily the best representation.)

## Question 5 (5 points)

See the file `nytcounts.university_cat_dog`, which contains context count vectors for three words: “dog”, “cat”, and “university”. These are immediate left and right contexts from a New York Times corpus. You can open the file in a text editor since it’s quite small. 

Write a function which takes context count dictionaries of two words and calculates cosine similarity between these two words. The function should return a number beween 0 and 1.  Briefly comment on whether the relative simlarities make sense.



In [2]:
import distsim; reload(distsim)
word_to_ccdict = distsim.load_contexts("nytcounts.university_cat_dog")



print distsim.calculate_cosine('dog','cat',word_to_ccdict)
print distsim.calculate_cosine('cat','dog',word_to_ccdict)
print distsim.calculate_cosine('university','cat',word_to_ccdict)
print distsim.calculate_cosine('dog','university',word_to_ccdict)



# write code here to show output (i.e. cosine similarity between these words.)
# We encourage you to write other functions in distsim.py itself. 

file nytcounts.university_cat_dog has contexts for 3 words
0.966891672715
0.966891672715
0.660442421144
0.659230248969


**Write your response here:**
Yes the similarities make sense. The words 'dog' and 'cat' both represent animals and hence we can expect a higher similarity(closer to 1 than 0) compared to 'university' and 'dog' or  'university' and 'cat'. The latter pair doesn't have a common word or a sense like animals.





## Question 6 (20 points)

Explore similarities in `nytcounts.4k`, which contains context counts for about 4000 words in a sample of New York Times articles. The news data was lowercased and URLs were removed. The context counts are for the 2000 most common words in twitter, as well as the most common 2000 words in the New York Times. (But all context counts are from New York Times.) The context counts only contain contexts that appeared for more than one word.  The file has three tab-separate fields: the word, its count, and a JSON-encoded dictionary of its context counts.  You'll see it's just counts of the immediate left/right neighbors.

Choose **six** words. For each, show the output of 20 nearest words (use cosine similarity as distance metric). Comment on whether the output makes sense. Comment on whether this approach to distributional similarity makes more or less sense for certain terms.
Four of your words should be:

 * a name (for example: person, organization, or location)
 * a common noun
 * an adjective
 * a verb

You may also want to try exploring further words that are returned from a most-similar list from one of these. You can think of this as traversing the similarity graph among words.

*Implementation note:* 
On my laptop it takes several hundred MB of memory to load it into memory from the `load_contexts()` function. If you don’t have enough memory available, your computer will get very slow because the OS will start swapping. If you have to use a machine without that much memory available, you can instead implement in a streaming approach by using the `stream_contexts()` generator function to access the data; this lets you iterate through the data from disk, one vector at a time, without putting everything into memory. You can see its use in the loading function. (You could also alternatively use a key-value or other type of database, but that’s too much work for this assignment.)

In [5]:
import distsim; reload(distsim)
import operator
word_to_ccdict = distsim.load_contexts("nytcounts.4k")
###Provide your answer below; perhaps in another cell so you don't have to reload the data each time

file nytcounts.4k has contexts for 3648 words


In [26]:
words=word_to_ccdict.keys()
chosen_words=['three','disney','animal','screaming','japanese','run']
#chosen_words=['run']
for cword in chosen_words:
    print cword
    d={}
    for word in words:
        d[word]=distsim.calculate_cosine(cword,word,word_to_ccdict)
    d_sorted=sorted(d.items(),key=operator.itemgetter(1),reverse=True)
    for a in d_sorted[:20]:
        print a

three
('three', 0.9999999999999998)
('four', 0.9749468073763116)
('seven', 0.9421338296520655)
('two', 0.9389568595828977)
('eight', 0.9367590971990057)
('nine', 0.9157102516178117)
('five', 0.9008175401540782)
('six', 0.885848972136979)
('several', 0.8068347540165448)
('various', 0.5989985797397092)
('south', 0.5750889013435223)
('most', 0.5621031500111197)
('ten', 0.5550365263003226)
('corporate', 0.5537181603918324)
('many', 0.5522872039535461)
('other', 0.5472531121442308)
('japanese', 0.5449717815901273)
('early', 0.5445351060232407)
('american', 0.5414345684691162)
('western', 0.5322075315867072)
disney
('disney', 1.0)
('microsoft', 0.5673368282508056)
('google', 0.5657684330655411)
('sony', 0.5568953141803903)
('wal-mart', 0.5392594980908143)
('nbc', 0.5247451393254923)
('women', 0.5230723211787983)
('israel', 0.5133133542476026)
('iran', 0.5120532739846196)
('apple', 0.5118707851796734)
('ford', 0.5100990527639214)
('russia', 0.5013274583496)
('fifa', 0.49602403222221525)
('abc

## Question 7 (10 points)

In the next several questions, you'll examine similarities in trained word embeddings, instead of raw context counts.

See the file `nyt_word2vec.university_cat_dog`, which contains word embedding vectors pretrained by word2vec [1] for three words: “dog”, “cat”, and “university”, from the same corpus. You can open the file in a text editor since it’s quite small.

Write a function which takes word embedding vectors of two words and calculates cossine similarity between these 2 words. The function should return a number beween -1 and 1. Briefly comment on whether the relative simlarities make sense. 

*Implementation note:*
Notice that the inputs of this function are numpy arrays (v1 and v2). If you are not very familiar with the basic operation in numpy, you can find some examples in the basic operation section here:
https://docs.scipy.org/doc/numpy-dev/user/quickstart.html

If you know how to use Matlab but haven't tried numpy before, the following link should be helpful:
https://docs.scipy.org/doc/numpy-dev/user/numpy-for-matlab-users.html

[1] Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." NIPS 2013.

In [7]:
import distsim; reload(distsim)

word_to_vec_dict = distsim.load_word2vec("nyt_word2vec.4k")
w=['dog','cat','university']
for word1 in w:
    for word2 in w:
        print word1,word2,distsim.calculate_cosine2(word_to_vec_dict[word1],word_to_vec_dict[word2],word_to_vec_dict)

# write code here to show output (i.e. cosine similarity between these words.)
# We encourage you to write other functions in distsim.py itself. 

dog dog 1.0
dog cat 0.827517295965
dog university -0.190753135501
cat dog 0.827517295965
cat cat 1.0
cat university -0.205394745036
university dog -0.190753135501
university cat -0.205394745036
university university 1.0


**Write your response here:**
Yes the similarities make sense. The words dog and cat which are more similar than university and any other pair of words. The closer the value is to 1, the more similar the words are. 1 means high similarity, -1 mean least similarity.



## Question 8 (20 points)

Repeat the process you did in the question 6, but now use dense vector from word2vec. Comment on whether the outputs makes sense. Compare the outputs of using nearest words on word2vec and the outputs on sparse context vector (so we suggest you to use the same words in question 6). Which method works better on the query words you choose. Please brief explain why one method works better than other in each case.

Not: we used the default parameters of word2vec in [gensim](https://radimrehurek.com/gensim/models/word2vec.html) to get word2vec word embeddings.

In [8]:
import distsim
word_to_vec_dict = distsim.load_word2vec("nyt_word2vec.4k")
###Provide your answer below

In [25]:
words=word_to_vec_dict.keys()
chosen_words=['screaming','japanese','run','three','disney','animal']
print chosen_words
for cword in chosen_words:
    d={}
    print "Word: ",cword
    for word in words:
        d[word]=distsim.calculate_cosine2(word_to_vec_dict[cword],word_to_vec_dict[word],word_to_vec_dict)
    d_sorted=sorted(d.items(),key=operator.itemgetter(1),reverse=True)
    for a in d_sorted[:20]:
        print a

['screaming', 'japanese', 'run', 'three', 'disney', 'animal']
Word:  screaming
('screaming', 1.0000000000000002)
('crying', 0.73219867891362722)
('laughing', 0.6858685239083866)
('smiling', 0.63642254060430448)
('naked', 0.63235707262200502)
('kissing', 0.63096127829583548)
('singing', 0.6185438401954475)
('drunk', 0.6180235507923999)
('tears', 0.61701756914507955)
('kisses', 0.60972622560174172)
('hanging', 0.60084804045142592)
('smiles', 0.59562358059815956)
('bored', 0.59508589672160095)
('walking', 0.59223123876312977)
('hugs', 0.5874100756973708)
('laughed', 0.58376950028770247)
('sleeping', 0.58045970820584958)
('dirty', 0.57537731332430064)
('dancing', 0.57437367487493973)
('crazy', 0.57376537236881631)
Word:  japanese
('japanese', 1.0)
('italian', 0.75305258552760623)
('chinese', 0.73367506007804639)
('german', 0.72262678546585168)
('french', 0.71189887975571409)
('spanish', 0.70850130994606419)
('british', 0.69558709025707188)
('russian', 0.68758910212174851)
('indian', 0.6707

The method with dense vectors from word2vec works better. This is because the quantity represented by word2vec better represents a 'distance' metric in the context of words. Whereas frequency of words is a poor metric. Cosine measures the similarity by seeing how similar the metrics of 'distance' is.

three :Both methods produce good results but the second method produces better results which includes digits as well. Hence it is better.

disney :Wrod2ved produces TV channel names as well which better qualifies what disney actually represents.

animal :Both methods produce comparable results.

Screaming: Word2vec works well and the contect counts do not produce results that are even half as good. This is because the fequency count of 'screaming' doesn't well represent a characteristic of the word which can be compared using cosine.

Japanese: Both methods produce good results but the second method produces better results because of a similar reason as above.

run :Word2vec produces more results which a verbs similar to running. Context counts produces main verbs but they are not all the same or similar to running.

## Question 9 (15 points)
An interesting thing to try with word embeddings is analogical reasoning tasks. In the following example, it's intended to solve the analogy question "king is to man as what is to woman?", or in SAT-style notation,

king : man :: ____ : woman

Some research has proposed to use additive operations on word embeddings to solve the analogy: take the vector $(v_{king}-v_{man}+v_{woman})$ and find the most-similar word to it.  One way to explain this idea: if you take "king", get rid of its attributes/contexts it shares with "man", and add in the attributes/contexts of "woman", hopefully you'll get to a point in the space that has king-like attributes but the "man" ones replaced with "woman" ones.

Show the output for 20 closest words you get by trying to solve that analogy with this method.  Did it work?

Please come up with another analogical reasoning task (another triple of words), and output the answer using the same method. Comment on whether the output makes sense. If the output makes sense, explain why we can capture such relation between words using an unsupervised algorithm. Where does the information come from? On the other hand, if the output does not make sense, propose an explanation why the algorithm fails on this case.


Note that the word2vec is trained in an unsupervised manner just with distributional statistics; it is interesting that it can apparently do any reasoning at all.  For a critical view, see [Linzen 2016](http://www.aclweb.org/anthology/W/W16/W16-2503.pdf).



In [14]:
# Write code to show output here.
import numpy as np
a=word_to_vec_dict['king']
b=word_to_vec_dict['man']
c=word_to_vec_dict['woman']
final=np.add(np.subtract(a,b),c)
d={}
for word in words:
    d[word]=distsim.calculate_cosine2(final,word_to_vec_dict[word],word_to_vec_dict)
d_sorted=sorted(d.items(),key=operator.itemgetter(1),reverse=True)
for a in d_sorted[:20]:
    print a

('king', 0.73899645785637358)
('queen', 0.72502863198579415)
('princess', 0.57790010340102305)
('prince', 0.56696239241698621)
('lord', 0.53091939111112829)
('royal', 0.52020329686379785)
('woman', 0.51095444078929031)
('mary', 0.49769814628352971)
('mama', 0.49546963683192269)
('daughter', 0.4937579465656014)
('singer', 0.48983808201392826)
('kim', 0.48835469524343844)
('elizabeth', 0.48248484340535652)
('girl', 0.47733829480775564)
('grandma', 0.47699072668127218)
('sister', 0.47030437182473761)
('mother', 0.46942202883251716)
('clark', 0.46824004740972941)
('wedding', 0.4623362935603858)
('husband', 0.45685118817885456)


In [17]:
# Write code to show output here.
a=word_to_vec_dict['night']
b=word_to_vec_dict['moon']
c=word_to_vec_dict['sun']
final=np.add(np.subtract(a,b),c)
d={}
for word in words:
    d[word]=distsim.calculate_cosine2(final,word_to_vec_dict[word],word_to_vec_dict)
d_sorted=sorted(d.items(),key=operator.itemgetter(1),reverse=True)
for a in d_sorted[:20]:
    print a

('night', 0.9220362189833351)
('morning', 0.75921385759763038)
('evening', 0.74575515718484331)
('afternoon', 0.74016504369220248)
('day', 0.62556316875232776)
('midnight', 0.58510039759843402)
('weekend', 0.57986869129579033)
('winter', 0.57258162994149642)
('minute', 0.55953118512125166)
('spring', 0.5455509774664643)
('saturday', 0.54399630990896375)
('sunday', 0.54231607557199923)
('session', 0.53158190145575868)
('sun', 0.52137887354492074)
('summer', 0.51807871602735212)
('season', 0.51642351240675521)
('nights', 0.49947684815416871)
('week', 0.49749944465101126)
('fall', 0.48959808034124258)
('pitch', 0.48491948584597788)


### Answer
Yes the method did work. And this makes sense because you are trying to add the qualities(distance) of woman and subtracting the qualities(distance) of man in the word vector space. This will probably not work if the context counts were used.

Proposed analogy-> night : moon :: ____ : sun

Proposed answer -> morning or day

The result of this analogy is morning and this is correct. It works because all the words night, moon and day are in the data and the analogy is straightforward. What we're trying to do here is capturing the essence of the the word pair night and moon and establishing a similar essence in another pair ? and sun.